# **Assignment 2**
# House Price Regression
19BCP068
KHUSHI KATARIYA

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
train= pd.read_csv("/content/drive/My Drive/train.csv")
test= pd.read_csv("/content/drive/My Drive/test.csv")

In [ ]:
train.shape

(1460, 81)

In [ ]:
test.shape

(1459, 80)

In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
# Missing Data or Null

train.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [ ]:
# Missing Data or Null

test.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [ ]:
# Drop Id 

train.drop(['Id'], axis = 1, inplace = True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [ ]:
# Split categorical and numerical variable

num_cols = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1',
            'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF',
            '2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath',
            'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',
            'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt',
            'GarageCars','GarageArea','WoodDeckSF','OpenPorchSF',
            'EnclosedPorch','3SsnPorch','ScreenPorch',
            'PoolArea','MiscVal','YearBuilt','YearRemodAdd','SalePrice',
            'MoSold','YrSold']

obj_cols = [col for col in train.columns 
                if train[col].dtype == "object" or
                   col not in num_cols]

In [ ]:
ordinal_cols = ['LotShape','LandSlope','OverallQual','OverallCond',
                'ExterQual','ExterCond','BsmtQual','BsmtCond',
                'BsmtExposure','BsmtFinType1','BsmtFinType2',
                'HeatingQC','KitchenQual','FireplaceQu','GarageQual',
                'GarageCond','PoolQC']

ordinal_cols

['LotShape',
 'LandSlope',
 'OverallQual',
 'OverallCond',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'PoolQC']

In [ ]:
nominal_cols = [col for col in obj_cols
                if col not in ordinal_cols]

nominal_cols

['MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LandContour',
 'Utilities',
 'LotConfig',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'Heating',
 'CentralAir',
 'Electrical',
 'Functional',
 'GarageType',
 'GarageFinish',
 'PavedDrive',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [ ]:
# NAs that should be None

train['Alley'] = train['Alley'].astype(object).replace(np.nan, 'None')
train['MasVnrType'] = train['MasVnrType'].astype(object).replace(np.nan, 'None')
train['BsmtQual'] = train['BsmtQual'].astype(object).replace(np.nan, 'None')
train['BsmtCond'] = train['BsmtCond'].astype(object).replace(np.nan, 'None')
train['BsmtExposure'] = train['BsmtExposure'].astype(object).replace(np.nan, 'None')
train['BsmtFinType1'] = train['BsmtFinType1'].astype(object).replace(np.nan, 'None')
train['BsmtFinType2'] = train['BsmtFinType2'].astype(object).replace(np.nan, 'None')
train['FireplaceQu'] = train['FireplaceQu'].astype(object).replace(np.nan, 'None')
train['GarageFinish'] = train['GarageFinish'].astype(object).replace(np.nan, 'None')
train['GarageType'] = train['GarageType'].astype(object).replace(np.nan, 'None')
train['GarageQual'] = train['GarageQual'].astype(object).replace(np.nan, 'None')
train['GarageCond'] = train['GarageCond'].astype(object).replace(np.nan, 'None')
train['PoolQC'] = train['PoolQC'].astype(object).replace(np.nan, 'None')
train['Fence'] = train['Fence'].astype(object).replace(np.nan, 'None')
train['MiscFeature'] = train['MiscFeature'].astype(object).replace(np.nan, 'None')

In [ ]:
# NAs that should be None

test['Alley'] = test['Alley'].astype(object).replace(np.nan, 'None')
test['MasVnrType'] = test['MasVnrType'].astype(object).replace(np.nan, 'None')
test['BsmtQual'] = test['BsmtQual'].astype(object).replace(np.nan, 'None')
test['BsmtCond'] = test['BsmtCond'].astype(object).replace(np.nan, 'None')
test['BsmtExposure'] = test['BsmtExposure'].astype(object).replace(np.nan, 'None')
test['BsmtFinType1'] = test['BsmtFinType1'].astype(object).replace(np.nan, 'None')
test['BsmtFinType2'] = test['BsmtFinType2'].astype(object).replace(np.nan, 'None')
test['FireplaceQu'] = test['FireplaceQu'].astype(object).replace(np.nan, 'None')
test['GarageFinish'] = test['GarageFinish'].astype(object).replace(np.nan, 'None')
test['GarageType'] = test['GarageType'].astype(object).replace(np.nan, 'None')
test['GarageQual'] = test['GarageQual'].astype(object).replace(np.nan, 'None')
test['GarageCond'] = test['GarageCond'].astype(object).replace(np.nan, 'None')
test['PoolQC'] = test['PoolQC'].astype(object).replace(np.nan, 'None')
test['Fence'] = test['Fence'].astype(object).replace(np.nan, 'None')
test['MiscFeature'] = test['MiscFeature'].astype(object).replace(np.nan, 'None')

In [ ]:
# Replace -1 from Electrical

train['Electrical'] = train['Electrical'].astype(object).replace('-1', 'None')
test['Electrical'] = test['Electrical'].astype(object).replace('-1', 'None')

Outliers

In [ ]:
from sklearn.model_selection import train_test_split

# Separate target from predictors
y = train['SalePrice']
X = train.drop(['SalePrice'], axis=1)

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, 
                                                      test_size=0.2,
                                                      random_state=42)

In [ ]:
# Impute Missing Values for Numeric Columns

from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
impute_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
impute_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
impute_X_train.columns = X_train.columns
impute_X_valid.columns = X_valid.columns

X_train = impute_X_train
X_valid = impute_X_valid

In [ ]:
# Transform Categorical (Nominal) Columns

from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[nominal_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[nominal_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(nominal_cols, axis=1)
num_X_valid = X_valid.drop(nominal_cols, axis=1)

# Add one-hot encoded columns to numerical features
X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [ ]:
# Transform Categorical (Ordinal) Columns

from sklearn.preprocessing import OrdinalEncoder

LotShape_cat = ['Reg','IR1','IR2','IR3']
LandSlope_cat = ['Gtl','Mod','Sev']
OverallQual_cat = [1,2,3,4,5,6,7,8,9,10]
OverallCond_cat = [1,2,3,4,5,6,7,8,9,10]
ExterQual_cat = ['Ex','Gd','TA','Fa','Po']
ExterCond_cat = ['Ex','Gd','TA','Fa','Po']
BsmtQual_cat = ['Ex','Gd','TA','Fa','Po','None']
BsmtCond_cat = ['Ex','Gd','TA','Fa','Po','None']
BsmtExposure_cat = ['Gd','Av','Mn','No','None']
BsmtFinType1_cat = ['GLQ','ALQ','BLQ','Rec','LwQ','Unf','None']
BsmtFinType2_cat = ['GLQ','ALQ','BLQ','Rec','LwQ','Unf','None']
HeatingQC_cat = ['Ex','Gd','TA','Fa','Po']
KitchenQual_cat = ['Ex','Gd','TA','Fa','Po']
FireplaceQu_cat = ['Ex','Gd','TA','Fa','Po','None']
GarageQual_cat = ['Ex','Gd','TA','Fa','Po','None']
GarageCond_cat = ['Ex','Gd','TA','Fa','Po','None']
PoolQC_cat = ['Ex','Gd','TA','Fa','None']

# Apply ordinal encoder to each column with categorical data
OR_encoder = OrdinalEncoder(categories=[LotShape_cat, LandSlope_cat, OverallQual_cat,
                                       OverallCond_cat, ExterQual_cat, ExterCond_cat,
                                       BsmtQual_cat, BsmtCond_cat, BsmtExposure_cat,
                                       BsmtFinType1_cat, BsmtFinType2_cat, HeatingQC_cat,
                                       KitchenQual_cat, FireplaceQu_cat, GarageQual_cat,
                                       GarageCond_cat, PoolQC_cat])
OR_cols_train = pd.DataFrame(OR_encoder.fit_transform(X_train[ordinal_cols]))
OR_cols_valid = pd.DataFrame(OR_encoder.transform(X_valid[ordinal_cols]))

# Removed index; put it back
OR_cols_train.index = X_train.index
OR_cols_valid.index = X_valid.index

# Remove categorical columns
num_X_train = X_train.drop(ordinal_cols, axis=1)
num_X_valid = X_valid.drop(ordinal_cols, axis=1)

# Add ordinal encoded columns to numerical features
X_train = pd.concat([num_X_train, OR_cols_train], axis=1)
X_valid = pd.concat([num_X_valid, OR_cols_valid], axis=1)

Algorithm and Prediction


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

print('mae:',mean_absolute_error(y_valid, y_pred,))
print('mse:',mean_squared_error(y_valid, y_pred))
print('rmse:',mean_squared_error(y_valid, y_pred, squared=False))
print('target_mean:',y.mean())
print('target_std:',np.std(y))

mae: 19947.051675342667
mse: 936363714.9564315
rmse: 30600.060701842267
target_mean: 180921.19589041095
target_std: 79415.29188606751
